In [1]:
# -------------------------------------------------------------------------------
# elftools example: dwarf_location_info.py
#
# Examine DIE entries which have either location list values or location
# expression values and decode that information.
#
# Location information can either be completely contained within a DIE
# (using 'DW_FORM_exprloc' in DWARFv4 or 'DW_FORM_block1' in earlier
# versions) or be a reference to a location list contained within
# the .debug_loc section (using 'DW_FORM_sec_offset' in DWARFv4 or
# 'DW_FORM_data4' / 'DW_FORM_data8' in earlier versions).
#
# The LocationParser object parses the DIE attributes and handles both
# formats.
#
# The directory 'test/testfiles_for_location_info' contains test files with
# location information represented in both DWARFv4 and DWARFv2 forms.
#
# Eli Bendersky (eliben@gmail.com)
# This code is in the public domain
#-------------------------------------------------------------------------------
from __future__ import print_function
import sys

# If pyelftools is not installed, the example can also run from the root or
# examples/ dir of the source distribution.
sys.path[0:0] = ['.', '..']

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)



def show_loclist(loclist, dwarfinfo, indent, cu_offset):
    """ Display a location list nicely, decoding the DWARF expressions
        contained within.
    """
    d = []
    for loc_entity in loclist:
        if isinstance(loc_entity, LocationEntry):
            d.append('%s <<%s>>' % (
                loc_entity,
                describe_DWARF_expr(loc_entity.loc_expr, dwarfinfo.structs, cu_offset)))
        else:
            d.append(str(loc_entity))
    return '\n'.join(indent + s for s in d)



In [16]:
def get_DIE_at_offset(CU, offset):
#         print('****'*5,CU.cu_offset)
        for die in CU.iter_DIEs():
            if die.offset == CU.cu_offset+offset:
                return die 
        return None

def get_type_name(CU, offset):#get_DIE_at_offset(CU,attr.value)
    die = get_DIE_at_offset(CU, offset)
    
    if die.tag == 'DW_TAG_pointer_type':
        for _attr in die.attributes.values():
            if _attr.name== "DW_AT_type":
                return "*"+get_type_name(CU,_attr.value) 
        
    
    for attr in die.attributes.values():
        if attr.name== "DW_AT_name":
            return attr.value.decode("utf-8")
    
    


In [27]:
from collections import defaultdict
def line_entry_mapping(line_program):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state or lpe.state.file == 0:
            continue
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1

    # for filename, lpe_count in filename_map.items():
    #     print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename(line_program, file_index):

    lp_header = line_program.header
    file_entries = lp_header["file_entry"]
    
    # print('lp_header', lp_header, '\n\n')
    # print("file_entries", file_entries,'\n\n__________________________________________\n\n')

    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode()

    directory = lp_header["include_directory"][dir_index - 1]
    return posixpath.join(directory, file_entry.name).decode()

In [36]:

FUNC_PARAMS = {}


def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        # get_dwarf_info returns a DWARFInfo context object, which is the
        # starting point for all DWARF-based processing in pyelftools.
        dwarfinfo = elffile.get_dwarf_info()

        # The location lists are extracted by DWARFInfo from the .debug_loc
        # section, and returned here as a LocationLists object.
        location_lists = dwarfinfo.location_lists()

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        # Create a LocationParser object that parses the DIE attributes and
        # creates objects representing the actual location information.
        loc_parser = LocationParser(location_lists)
        
        section_offset = dwarfinfo.debug_info_sec.global_offset
        # Offset of the .debug_info section in the stream
        

        
        for CU in dwarfinfo.iter_CUs():
            
                    #get source file name
            line_program = dwarfinfo.line_program_for_CU(CU)
            filename_map = line_entry_mapping(line_program)
            print(list(filename_map.items())[0])
            # CU_dictionary_key = filename_map+"_"+str(CU.cu_offset)
            
            print('  Found a compile unit at offset %s, length %s' % (
                CU.cu_offset, CU['unit_length']))

            # A CU provides a simple API to iterate over all the DIEs in it.
            die_depth = 0
            are_DIEs_of_function = False
            FUNC_name = ''
            for DIE in CU.iter_DIEs():
                
                # print(DIE)
                ###############################################
                #############   Prasing Function DIEs start ################
                if DIE.tag == 'DW_TAG_subprogram':
                    are_DIEs_of_function = True
    

                    for attr in DIE.attributes.values():
                        if attr.name == "DW_AT_name": #FUNC NAME
                            FUNC_name = attr.value.decode("utf-8")
                            print(FUNC_name)
                            
                            
                if DIE.tag == 'DW_TAG_formal_parameter':
                    tags = [attr.name for attr in DIE.attributes.values()]
                    if "DW_AT_name" in tags:
                        for attr in DIE.attributes.values():
                            
                            if attr.name == "DW_AT_name": # VAR NAME 
                                PARAM_name = attr.value.decode("utf-8")
                                print('------------> Param:  ',PARAM_name)
                                
                            if attr.name == "DW_AT_type":
                                print('------------------> type: ',get_type_name(CU,attr.value),'\n\n')
                                
                                
#                                 DW_TAG_variable
                if DIE.tag == 'DW_TAG_variable':

                    for attr in DIE.attributes.values():

                        if attr.name == "DW_AT_name": # VAR NAME 
                            PARAM_name = attr.value.decode("utf-8")
                            print('------------> Local: ',PARAM_name)

                        if attr.name == "DW_AT_type":
                            print('------------------> type: ',get_type_name(CU,attr.value),'\n\n')



                ###############################################
                #############  parsing  Function DIEs ends ################
                


                
                if DIE.is_null(): #https://chromium.googlesource.com/chromiumos/third_party/pyelftools/+/25a77f7738d7fe824f2ed4d33a123136b9d8e88a/scripts/readelf.py
                    are_DIEs_of_function = False
                    FUNC_name = None
                    
                    die_depth -= 1
                    continue
                if DIE.has_children:
                    die_depth += 1
                    
#             break



filename = './../../binaries/c_many/stacktest'
process_file(filename)

Processing file: ./../../binaries/c_many/stacktest
('stack.c', 21)
  Found a compile unit at offset 0, length 457
free
malloc
pop
------------> Param:   stk_ptr
------------------> type:  **stack 


------------> Local:  pop
------------------> type:  int 


------------> Local:  number
------------------> type:  int 


------------> Local:  stk
------------------> type:  *stack 


------------> Local:  tmp
------------------> type:  *stack 


push
------------> Param:   number
------------------> type:  int 


------------> Param:   stk_ptr
------------------> type:  **stack 


------------> Local:  pop
------------------> type:  int 


------------> Local:  ming
------------------> type:  unsigned int 


------------> Local:  stk
------------------> type:  *stack 


------------> Local:  tmp
------------------> type:  *stack 


('main.c', 16)
  Found a compile unit at offset 461, length 350
printf
pop
add
push
main
------------> Local:  stk
------------------> type:  *stack 


------